# Point Cloud Data -> preprocessing_point_cloud_data(file_path)

In [1]:
# test
import numpy as np

file_path=r'.\data_road\training\velodyne\um_000000.bin'

x, y, z, r=np.fromfile(file_path, dtype=np.float32).reshape(-1, 4).T
a=np.sqrt(x*x+y*y+z*z)         # radial distance                  # 3
b=np.rad2deg(np.arctan2(y, x)) # polar angle   (horizontal angle) # 4
c=np.rad2deg(np.arccos(z/a))   # azimuth angle (vertical angle)   # 5

x=np.vstack([x, y, z, a, b, c, r]).T

# horizontal line grouping to [-45º, 45º)
x=x[(45>x[:, 4])&(x[:, 4]>=-45)]
# vertical line grouping to [min(x), max(x))
c_max, c_min=max(x[:, 5]), min(x[:, 5])

# horizontal grouping
tmp=np.linspace(-45, 45 , 180+1)
x=np.array([x[(tmp[i+1]>x[:, 4])&(x[:, 4]>=tmp[i])] for i in range(180)])
# vertical grouping
tmp=np.linspace(c_min, c_max, 64+1)
x=np.array([[j[(tmp[i+1]>j[:, 5])&(j[:, 5]>=tmp[i])] for i in range(64)] for j in x])

# sampling
x=np.array([[np.concatenate([i[np.argmin(i[:, 3])], i[np.argmax(i[:, 3])]]) if len(i)>0 else np.array([-1]*14) for i in j]for j in x])

# Interpolation
for i in range(180):
    for j in range(64):
        if(np.all(x[i][j]==np.array([-1]*14))):
            tmp=np.array([0.]*14)
            count=0
            if(j+1<64): count, tmp=count+1, tmp+x[i][j+1]
            if(j-1>=0): count, tmp=count+1, tmp+x[i][j-1]
            x[i][j]=tmp/count

In [30]:
# function
import numpy as np

def preprocessing_point_cloud_data(file_path):
    x, y, z, r=np.fromfile(file_path, dtype=np.float32).reshape(-1, 4).T
    a=np.sqrt(x*x+y*y+z*z)         # radial distance                  # 3
    b=np.rad2deg(np.arctan2(y, x)) # polar angle   (horizontal angle) # 4
    c=np.rad2deg(np.arccos(z/a))   # azimuth angle (vertical angle)   # 5

    x=np.vstack([x, y, z, a, b, c, r]).T

    # horizontal line grouping to [-45º, 45º)
    x=x[(45>x[:, 4])&(x[:, 4]>=-45)]
    # vertical line grouping to [min(x), max(x))
    c_max, c_min=max(x[:, 5]), min(x[:, 5])

    # horizontal grouping
    tmp=np.linspace(-45, 45 , 180+1)
    x=np.array([x[(tmp[i+1]>x[:, 4])&(x[:, 4]>=tmp[i])] for i in range(180)])
    # vertical grouping
    tmp=np.linspace(c_min, c_max, 64+1)
    x=np.array([[j[(tmp[i+1]>j[:, 5])&(j[:, 5]>=tmp[i])] for i in range(64)] for j in x])

    # sampling
    x=np.array([[np.concatenate([i[np.argmin(i[:, 3])], i[np.argmax(i[:, 3])]]) if len(i)>0 else np.array([-1]*14) for i in j]for j in x])
    
    # Interpolation
    for i in range(180):
        for j in range(64):
            if(np.all(x[i][j]==np.array([-1]*14))):
                tmp=np.array([0.]*14)
                count=0
                if(j+1<64): count, tmp=count+1, tmp+x[i][j+1]
                if(j-1>=0): count, tmp=count+1, tmp+x[i][j-1]
                x[i][j]=tmp/count
                
    return x

# Ground Truth -> preprocessing_ground_truth(pcd_file_path, gt_file_path, calib_file_path)

In [3]:
# test
import matplotlib.pyplot as plt
from BirdsEyeView import readKittiCalib

def lidar2camera(x, calibration):
    x=calibration.dot(np.append(x, 1))
    return x[: 2]/x[2]
def get_binary_matrix(ground_truth, x):
    if(np.all((x>=0)&(ground_truth.shape>x))):
        return ground_truth[x.astype('int')[0]][x.astype('int')[1]]
    else: return 0

# get point could data
pcd_file_path=r'.\data_road\training\velodyne\um_000000.bin'
x=preprocessing_point_cloud_data(pcd_file_path)

# get ground truth data
gt_file_path=r'.\data_road\training\gt_image_2\road\um_road_000000.png'
gt=plt.imread(gt_file_path)[:, :, 2]

# get calibration matrix
calib_file_path=r'.\data_road\training\calib\um_000000.txt'
calib=readKittiCalib(calib_file_path)
P2, R0, T=np.array(calib['P2']).reshape(3, 4), np.array(calib['R0_rect']).reshape(3, 3), np.array(calib['Tr_velo_to_cam']).reshape(3, 4)
calibration =P2.dot(np.vstack([R0.dot(T), [0, 0, 0, 1]]))

# convert to spherical coordinates
graph=np.array([[get_binary_matrix(gt, lidar2camera(x[j][i][: 3], calibration)[:: -1])*get_binary_matrix(gt, lidar2camera(x[j][i][7: 10], calibration)[:: -1]) for i in range(64)] for j in range(180)])

In [4]:
# function
import matplotlib.pyplot as plt
from BirdsEyeView import readKittiCalib

def preprocessing_ground_truth(pcd_file_path, gt_file_path, calib_file_path):
    def lidar2camera(x, calibration):
        x=calibration.dot(np.append(x, 1))
        return x[: 2]/x[2]
    def get_binary_matrix(ground_truth, x):
        if(np.all((x>=0)&(ground_truth.shape>x))):
            return ground_truth[x.astype('int')[0]][x.astype('int')[1]]
        else: return 0

    # get point could data
    x=preprocessing_point_cloud_data(pcd_file_path)

    # get ground truth data
    gt=plt.imread(gt_file_path)[:, :, 2]

    # get calibration matrix
    calib=readKittiCalib(calib_file_path)
    P2, R0, T=np.array(calib['P2']).reshape(3, 4), np.array(calib['R0_rect']).reshape(3, 3), np.array(calib['Tr_velo_to_cam']).reshape(3, 4)
    calibration =P2.dot(np.vstack([R0.dot(T), [0, 0, 0, 1]]))

    # convert to spherical coordinates
    graph=np.array([[get_binary_matrix(gt, lidar2camera(x[j][i][: 3], calibration)[:: -1])*get_binary_matrix(gt, lidar2camera(x[j][i][7: 10], calibration)[:: -1]) for i in range(64)] for j in range(180)])
    
    return graph

# test

In [5]:
import os

file_path_list_calib    =[os.path.join(r'.\data_road\training\calib', i)           for i in os.listdir(r'.\data_road\training\calib')]
file_path_list_gt_image2=[os.path.join(r'.\data_road\training\gt_image_2\road', i) for i in os.listdir(r'.\data_road\training\gt_image_2\road')]
file_path_list_velodyne =[os.path.join(r'.\data_road\training\velodyne', i)        for i in os.listdir(r'.\data_road\training\velodyne')]

X=np.array([preprocessing_point_cloud_data(file_path_list_velodyne[i]) for i in range(289)])
Y=np.array([preprocessing_ground_truth(file_path_list_velodyne[i], file_path_list_gt_image2[i], file_path_list_calib[i]) for i in range(289)])

In [6]:
np.save(r'./X.npy', X)
np.save(r'./Y.npy', Y)

In [5]:
import os

file_path_list_calib    =[os.path.join(r'.\data_road\testing\calib', i)           for i in os.listdir(r'.\data_road\testing\calib')]
# file_path_list_gt_image2=[os.path.join(r'.\data_road\testing\gt_image_2\road', i) for i in os.listdir(r'.\data_road\testing\gt_image_2\road')]
file_path_list_velodyne =[os.path.join(r'.\data_road\testing\velodyne', i)        for i in os.listdir(r'.\data_road\testing\velodyne')]

X=np.array([preprocessing_point_cloud_data(file_path_list_velodyne[i]) for i in range(290)])
Y=np.array([preprocessing_ground_truth(file_path_list_velodyne[i], file_path_list_gt_image2[i], file_path_list_calib[i]) for i in range(290)])

NameError: name 'file_path_list_gt_image2' is not defined